In [2]:
import os

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from dotenv import load_dotenv

In [7]:
load_dotenv()

True


### Loading the model and testing with the simple message

In [10]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [14]:
model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.7, convert_system_message_to_human=True)

In [15]:
message = [SystemMessage(content="Hi you are a nice bot"),HumanMessage(content="Hi i am Asif")]

In [23]:
response_1 = model.invoke(message)

In [24]:
response_1

AIMessage(content="Hello Asif, it's nice to meet you. I am happy to assist you with any questions you may have.", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-a5c5fb9f-bdcc-4cd7-9584-c381d5bc734c-0', usage_metadata={'input_tokens': 12, 'output_tokens': 25, 'total_tokens': 37})

In [25]:
response_1.content

"Hello Asif, it's nice to meet you. I am happy to assist you with any questions you may have."

#### Output parser

In [21]:
from langchain_core.output_parsers import StrOutputParser

In [22]:
parser = StrOutputParser()

In [26]:
parser.invoke(model.invoke(message))

'Thank you for the compliment. My name is Gemini. I am a multi-modal AI model, developed by Google. I am designed to understand and generate human language, and to answer questions and provide information to the best of my abilities. I am still under development, but I am learning new things every day.\n\nIs there anything I can help you with today?'

In [27]:
message_2 = [SystemMessage(content="Hi you are a nice urdu poet"),HumanMessage(content="can you write a poem for kids?")]

In [28]:
parser.invoke(model.invoke(message_2))

'**چاند تارے**\n\nآسماں پر چاند ہے، چمک رہا ہے،\nتارے جھلمل ہیں، مانو ہنس رہے ہیں۔\nچاند سورج کا بھائی ہے،\nتارے اس کے ساتھی ہیں۔\n\nچاند تارے سب مل کر،\nرات کو بناتے ہیں روشن۔\nبچے سو جائیں جلدی से،\nچاند تارے کہتے हैं।\n\nرات بھر چمکتے رہتے ہیں،\nصبح ہوتے ہی چھپ جاتے ہیں۔\nپھر آئیں گے کل رات کو،\nتمہیں پھر سے خواب دکھائیں گے۔'

#### chaining with LCEL

In [31]:
chain_1 = model | parser
chain_1.invoke(message)

"Hi Asif, it's nice to meet you. I'm Gemini, a multi-modal AI model, developed by Google. I'm designed to assist and provide information to the best of my abilities. Is there anything I can help you with today?"

In [32]:
chain_2 = model | parser
chain_1.invoke(message_2)

'**بچوں کا نغمہ**\n\nہم بچے ہیں چھوٹے چھوٹے\nکھیلنا ہمارا کام ہے\nہنستے کھیلتے گاتے ہیں\nیہی ہمارا کام ہے\n\n(کورس)\nہم ہیں بچے ہم ہیں بچے\nہماری دنیا ہے رنگین\nہماری خوشیوں میں کوئی غم نہیں\nہمارا بچپن ہے سنہریں\n\nپڑھنا لکھنا بھی ہم کو آتا ہے\nپر کھیلنا ہمیں سب سے پیارا ہے\nگڑیا گھوڑے، پتنگ اڑانا\nیہی ہے ہمارا شوق سب سے بڑا\n\n(کورس)\nہم ہیں بچے ہم ہیں بچے\nہماری دنیا ہے رنگین\nہماری خوشیوں میں کوئی غم نہیں\nہمارا بچپن ہے سنہریں'

### Prompt templating

In [36]:
from langchain.prompts import ChatPromptTemplate

In [37]:
prompt_template = ChatPromptTemplate.from_messages([
    "system","translate the following into {language}",
    "user","{text}"]
)

In [38]:
prompt = prompt_template.invoke({"language":"italian","text":"GenAI is the future of tech we must all learn"})

In [45]:
print(prompt)

messages=[HumanMessage(content='system'), HumanMessage(content='translate the following into italian'), HumanMessage(content='user'), HumanMessage(content='GenAI is the future of tech we must all learn')]


In [41]:
response = parser.invoke(model.invoke(prompt))

In [43]:
response

'GenAI è il futuro della tecnologia che tutti dobbiamo imparare'

In [46]:
prompt_2= prompt_template.invoke({"language":"german","text":"GenAI is the future of tech we must all learn"})

In [48]:
parser.invoke(model.invoke(prompt_2))

'GenAI ist die Zukunft der Technologie, die wir alle erlernen müssen'

## Applying chaining(LCEL)

In [55]:
chain_1 = prompt_template | model | parser

In [59]:
chain_1.invoke({"language":"pashto","text":"Aamir the great boy gonna get to married soon but he did not invite me so sad although he is my good friend i dont understand why he ignore me, i dont like this really"})

'اعظم امیر لوی هلک چټلې واده ته روان دی خو هغه ما ته بلنه نه ده ورکړه ډېره اندېښه ده حال دا چې هغه زه ښه ملګری یم زه نه پوهیږم ولې هغه ما ته پام نه دی ورکړی واقعاً ما ته دا نه ښکاري'

### Agents

In [131]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [132]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage

In [133]:
memory = MemorySaver()

In [134]:
search = TavilySearchResults(max_results=2)

In [127]:
tools = [search]

In [128]:
agent_executor = create_react_agent(model,tools,checkpointer=memory)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [129]:
config = {"configurable":{"thread_id": "abc130"}}

In [135]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="Hi i am Asif and live in Gilgit baltistan, pakistan")]},config):
    print(chunk)

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavilly_search_results_json', 'arguments': '{"query": "Is Gilgit Baltistan, Pakistan a province?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-f67a5e99-fd08-454c-8bad-d92c415cceac-0', tool_calls=[{'name': 'tavilly_search_results_json', 'args': {'query': 'Is Gilgit Baltistan, Pakistan a province?'}, 'id': '2c56fbe2-f5b5-4252-b59b-495335cf6c75', 'type': 'tool_call'}], usage_metadata={'input_tokens': 202, 'output_tokens': 31, 'total_tokens':

In [124]:
for chunk in agent_executor.stream({"messages":[HumanMessage(content="Can you tell me the weather where i live?")]},config=config):
    print(chunk)

{'agent': {'messages': [AIMessage(content='I do not have access to real-time information and cannot provide you with the weather conditions for your location.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-8677a082-9a55-458e-bc76-d914874683fa-0', usage_metadata={'input_tokens': 203, 'output_tokens': 22, 'total_tokens': 225})]}}
